In [22]:
import numpy as np
import pandas as pd

# Load the data files
x_train = np.load('data/X_train.npy')
x_test = np.load('data/X_test.npy')

# Checking the shapes of the datasets
x_train_shape = x_train.shape
x_test_shape = x_test.shape

x_train_shape, x_test_shape

((323, 40, 100), (54, 40, 100))

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Нормализация данных (StandardScaler делает нормализацию на основе среднего и стандартного отклонения)
scaler = StandardScaler()

# Применяем нормализацию ко всем временным шагам и датчикам обучающего набора
# Форма данных (наблюдения, датчики, время), поэтому нужно применить нормализацию вдоль датчиков (ось 1)
x_train_reshaped = x_train.reshape(-1, x_train.shape[1])
x_train_scaled = scaler.fit_transform(x_train_reshaped).reshape(x_train.shape)

# Аналогично для тестовых данных
x_test_reshaped = x_test.reshape(-1, x_test.shape[1])
x_test_scaled = scaler.transform(x_test_reshaped).reshape(x_test.shape)

# Разделение на тренировочный и валидационный набор (например, 80% для обучения, 20% для валидации)
x_train_split, x_val_split = train_test_split(x_train_scaled, test_size=0.2, random_state=42)

# Выводим размерности, чтобы проверить корректность обработки
print("Размеры после нормализации:")
print("Тренировочный набор:", x_train_split.shape)
print("Валидационный набор:", x_val_split.shape)
print("Тестовый набор:", x_test_scaled.shape)

Размеры после нормализации:
Тренировочный набор: (258, 40, 100)
Валидационный набор: (65, 40, 100)
Тестовый набор: (54, 40, 100)


In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

# Функция для вычисления F1-скора
def f1_score(y_true, y_pred):
    # Преобразуем вероятности предсказаний в бинарные метки
    y_pred_binary = K.round(y_pred)

    # Вычисляем True Positives, False Positives и False Negatives
    tp = K.sum(K.cast(y_true * y_pred_binary, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred_binary, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred_binary), 'float'), axis=0)

    # Вычисляем Precision и Recall
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    # Вычисляем F1-скор
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    
    # Усредняем по классам
    f1 = K.mean(f1)
    return f1

# Транспонирование данных (наблюдения, временные шаги, датчики)
x_train_scaled = np.transpose(x_train_scaled, (0, 2, 1))
x_test_scaled = np.transpose(x_test_scaled, (0, 2, 1))

# Создаем модель
input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])  # Форма входных данных (100 временных шагов, 40 датчиков)

model = Sequential()

# Первый LSTM слой
model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
model.add(Dropout(0.3))

# Второй LSTM слой
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.3))

# Полносвязный слой
model.add(Dense(64, activation='relu'))

# Выходной слой
model.add(Dense(n_classes, activation='softmax'))

# Компиляция модели с метрикой F1-score
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_score])

# Разделение данных на обучающие и валидационные выборки
x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train_scaled, y_train_encoded, test_size=0.2, random_state=42)

# Обучение модели
history = model.fit(x_train_split, y_train_split, 
                    validation_data=(x_val_split, y_val_split),
                    epochs=50, batch_size=32)

# Предсказания для тестового набора
y_pred_test = model.predict(x_test_scaled)

# Преобразование вероятностей в метки классов
y_pred_classes = np.argmax(y_pred_test, axis=-1)

# Выводим предсказанные классы для тестового набора
print("Предсказанные классы для тестового набора:", y_pred_classes)


Epoch 1/50


c:\Study_projects_part_2\Motorica_kaggle\mototica_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 127ms/step - accuracy: 0.2950 - f1_score: 0.0017 - loss: 2.1114 - val_accuracy: 0.5385 - val_f1_score: 0.0635 - val_loss: 1.6538
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5622 - f1_score: 0.0703 - loss: 1.6316 - val_accuracy: 0.5385 - val_f1_score: 0.0660 - val_loss: 1.6256
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.5444 - f1_score: 0.0583 - loss: 1.6526 - val_accuracy: 0.5385 - val_f1_score: 0.0669 - val_loss: 1.6487
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5517 - f1_score: 0.0740 - loss: 1.6211 - val_accuracy: 0.5385 - val_f1_score: 0.0671 - val_loss: 1.5967
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.5666 - f1_score: 0.0724 - loss: 1.5185 - val_accuracy: 0.5385 - val_f1_score: 0.0628 - val_loss: 1.5246
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5465 - f1_score: 0.0667 - loss: 1.5038 - val_accuracy: 0.5385 - val_f1_score: 0.0695 - val_loss: 1.4800
Epoch 7/

In [29]:
import pandas as pd
import numpy as np

# 1. Получение предсказаний
y_pred_test = model.predict(x_test_scaled)

# 2. Преобразование предсказаний в метки классов
y_pred_classes = np.argmax(y_pred_test, axis=-1)  # Метки классов

# 3. Форматирование предсказаний для соответствия файлу sample_submission.csv
n_samples = y_pred_classes.shape[0]  # Количество тестовых образцов
n_timesteps = y_pred_classes.shape[1]  # Количество временных шагов для каждого образца

# Создаем списки для записи в submission
sample_ids = []
timestep_ids = []
predicted_classes = []

start_sample_index = 323  # Начинаем с 323, как указано

for sample in range(n_samples):
    for timestep in range(n_timesteps):
        sample_ids.append(start_sample_index + sample)  # Индекс текущего образца начиная с 323
        timestep_ids.append(timestep)  # Индекс текущего временного шага
        predicted_classes.append(y_pred_classes[sample, timestep])  # Предсказанный класс для данного временного шага

# 4. Создание DataFrame
submission_df = pd.DataFrame({
    "sample-timestep": [f"{sample}-{timestep}" for sample, timestep in zip(sample_ids, timestep_ids)],
    "class": predicted_classes
})

# 5. Сохранение предсказаний в файл submission.csv
submission_file_path = 'data/submission.csv'
submission_df.to_csv(submission_file_path, index=False)

print(f"Файл с предсказаниями сохранен как {submission_file_path}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Файл с предсказаниями сохранен как data/submission.csv
